# Import and Store NNDC Deposition Data - First Objective

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

### Setup

The [radlist](https://www.nndc.bnl.gov/radlist/radlist2.jsp) API that parses the ENSDF Data needs the latter as its payload. To the user, merely knowing the nuclide and decay radiation parameters should be enough, as on [Decay Radiation Search](https://www.nndc.bnl.gov/nudat3/indx_dec.jsp), instead of copying values from one place to another.

On searching for the decay data of any nuclide on the webpage above, we get another page that holds the link to ENSDF data.

Therefore, we first parse this HTML response to fetch the `href` link to ENSDF Data using `BeautifulSoup`.

### Fetch the ENSDF Data Download Link

The payload for the API has been retrieved by inspecting the 'Network Activity' when sending a POST request to the URL.

In [2]:
def get_ensdf_link(nuclide):
    
    if not nuclide.strip():
        raise ValueError("Invalid Input")

    url = "https://www.nndc.bnl.gov/nudat3/dec_searchi.jsp"
    # default payload for the decay-search API
    data = {
        "spnuc": "name",
        "nuc": nuclide,
        "z": "",
        "a": "",
        "n": "",
        "zmin": "",
        "zmax": "",
        "amin": "",
        "amax": "",
        "nmin": "",
        "nmax": "",
        "evenz": "any",
        "evena": "any",
        "evenn": "any",
        "tled": "disabled",
        "tlmin": 0,
        "utlow": "FS",
        "tlmax": 1E10,
        "utupp": "GY",
        "dmed": "disabled",
        "dmn": "ANY",
        "rted": "disabled",
        "rtn": "ANY",
        "reed": "disabled",
        "remin": 0,
        "remax": 10000,
        "ried": "disabled",
        "rimin": 0,
        "rimax": 200,
        "ord": "zate",
        "out": "wp",
        "unc": "nds",
        "sub": "Search"
    }

    
    #Send POST request
    response = requests.post(url, data=data)
    response.raise_for_status()

    # Convert the response to string
    html_text = response.text

    # parse the HTML string through BeautifulSoup
    soup = BeautifulSoup(html_text, 'html.parser')
    
    links = soup.find_all('a')
    if(len(links)<2):
        raise ValueError("No datasets found within the specified search parameters")

    else:
        # The SECOND hyperlink on the webpage is ENSDF Download Link
        ensdf_link = links[1].get('href')
        return ensdf_link
    

### Fetch the ENSDF data with the link retrieved above through a GET request

In [3]:
def get_ensdf_data_from_link(ensdf_link):
    
    ensdf_link = "https://www.nndc.bnl.gov/nudat3/" + ensdf_link

    response = requests.get(ensdf_link)
    response.raise_for_status()

    soup = BeautifulSoup(response.text,'html.parser')
    ensdf_data = soup.find('pre').text

    return ensdf_data  

### Convert and Write to a JSON file after a POST request to 'radlist'

The ENSDF data is supposed to be such that there should be **no leading whitespace for A>99**, and **one leading whitespace for A<99**.

In [4]:
def count_digits(nuclide):
    count = 0
    for i in nuclide:
        count += 1 if i.isdigit() else 0
        
    return count

In [5]:
def get_json_from_ensdf(ensdf_data, nuclide):

    d = count_digits(nuclide)
    
    #Preparing the data to be parsed by 'radlist'.
    ensdf_data = " "*(3-d) + ensdf_data.strip()
    
    # Set the URL and data to send
    url = "https://www.nndc.bnl.gov/radlist/radlist2.jsp"
    data = {
        "mydata": ensdf_data,
        "format": "json",
        "submit": "Submit"
    }
 
    # Send the POST request. 'requests' converts data into the accepted Content-Type: x-www-form-urlencoded.
    response = requests.post(url, data=data)
    response.raise_for_status()

    #Write to file and Download
    with open(f'{nuclide}_json_data.json', 'w') as f:
        json.dump(response.json(), f)
    
    return response.json()

### Enter the nuclide

In [6]:
nuclide = "Co56"
if __name__ == "__main__":
    ensdf_link = get_ensdf_link(nuclide)
    ensdf_data = get_ensdf_data_from_link(ensdf_link)
    result = get_json_from_ensdf(ensdf_data, nuclide)
    
    

Since resulting JSON has several nested dictionaries and list of dictionaries, some of which may not be related to each other, I have therefore segregated a few dataframes based on the keys.

In [7]:
pd.DataFrame(result['parent'])

,a,decayMode,level,z,halflife,qValue,n,element
name,56,EC,NaN,27,NaN,NaN,29,Co
uncertainty,56,,NaN,27,26,20,29,Co
value,56,100,NaN,27,77.236,4566.0,29,Co
jpis,56,NaN,4+,27,NaN,NaN,29,Co
energy,56,NaN,"{'uncertainty': '', 'value': '0.0'}",27,NaN,NaN,29,Co
inSeconds,56,NaN,NaN,27,6673190.4,NaN,29,Co
units,56,NaN,NaN,27,D,NaN,29,Co


In [8]:
data = result['radiation']

pd.set_option("display.max_rows", None, "display.max_columns", None)


frames = [pd.DataFrame(data[v]) for v in list(data.keys())]
pd.concat(frames,keys = list(data.keys()))

dose radIntensityUncertainty radIntensity  \
betaPlusAV 0         0.121                      14         19.7   
electron   0       7.44E-4                       6        111.0   
           1      0.002605                       4         46.3   
           2        3.5E-7                      11       4.8E-5   
           3        6.6E-7                       9       8.4E-5   
           4   2.181799E-4                                0.026   
           5       1.92E-6                       7      1.98E-4   
           6       1.93E-5                      14      0.00187   
           7        2.5E-6                       5       2.1E-4   
           8       8.26E-5                      13      0.00671   
           9       4.40E-6                       9      3.26E-4   
           10      1.28E-5                       3       7.3E-4   
           11       5.4E-7                      14      2.76E-5   
           12      2.36E-6                       6      1.18E-4   
           13      5.86E-6                      10      2.89E-4   
           14      1.14E-5                      17      4.41E-4   
           15       8.1E-7                       7       2.7E-5   
           16      2.15E-6                       3       6.7E-5   
           17      5.02E-6                       7      1.55E-4   
           18      1.17E-6                       3       3.6E-5   
           19       3.7E-7                      19      1.07E-5   
betaPlus   0        3.7E-6                       8       0.0082   
           1        4.6E-8                      20       6.0E-5   
           2       0.00186                      20        1.041   
           3        1.7E-5                      13       0.0067   
           4         0.116                      14         18.4   
           5        0.0030                      17         0.25   
gamma      0        4.7E-6                       7         0.67   
           1       4.68E-4                       4          7.3   
           2        9.3E-4                       7         14.5   
           3       1.22E-4                       8         1.72   
           4        6.2E-5                       4         0.88   
           5       5.79E-5                       3       0.0220   
           6        9.9E-5                       3        0.024   
           7       2.63E-4                      20       0.0540   
           8                                     3           39   
           9        2.8E-4                       4        0.043   
           10      1.62E-4                       3        0.024   
           11     0.001400                       3        0.191   
           12     0.002448                       3        0.311   
           13     0.846261                              99.9399   
           14       4.2E-4                       3        0.049   
           15       6.5E-4                       3        0.073   
           16      0.01389                       6        1.421   
           17      0.00111                       4        0.111   
           18       0.1458                       4        14.05   
           19       6.0E-4                       4        0.055   
           20      0.00150                       3        0.132   
           21      0.00109                       6        0.094   
           22      0.02646                       6        2.252   
           23       5.9E-4                       5        0.049   
           24       0.8230                      12        66.46   
           25      2.54E-4                       7       0.0200   
           26     0.001635                      12       0.1224   
           27      0.05826                      12        4.283   
           28      0.00260                       4        0.180   
           29      0.00108                       4        0.074   
           30      0.00101                      19       0.0616   
           31       0.2730                       6        15.41   
  

# Tests

In [ ]:
!pip install ipytest

In [10]:
import pytest
import ipytest

ipytest.autoconfig()


VALID_VALUES = ["Am241", "   Am241", "Am241   ", "Am-241"]
INVALID_VALUES = ["", " ", "234", "C", "QWERTY", "-241Am"]


@pytest.mark.parametrize('valid_input', VALID_VALUES)
def test_valid_nuclide_input(valid_input):
    assert get_ensdf_link(valid_input) == "getdecaydataset.jsp?nucleus=237NP&dsid=241am a decay"


@pytest.mark.parametrize('invalid_input', INVALID_VALUES)
def test_invalid_nuclide_input(invalid_input):
    with pytest.raises(ValueError):
        get_ensdf_link(invalid_input)


In [11]:
def test_invalid_ensdf_link():
    with pytest.raises(requests.exceptions.HTTPError):
        get_ensdf_data_from_link("randomtestlink.jsp")

In [12]:
@pytest.mark.parametrize(["ensdf_data","nuclide"],[("","Co56"),("abcd","")])
def test_get_json_from_ensdf_invalid(ensdf_data,nuclide):
    with pytest.raises(json.JSONDecodeError):
        get_json_from_ensdf(ensdf_data,nuclide)

In [13]:
def test_get_json_from_ensdf_valid():
    ensdf_data = """ 56FE    56CO EC DECAY                 1990ME15,1988WA26,2008Dr0411NDS    201105
 56FE  H TYP=FUL$AUT=HUO JUNDE, HUO SU, YANG DONG$CIT=NDS 112, 1513 (2011)$
 56FE2 H CUT=29-Oct-2009$
 56FE c  Sources produced by {+56}Fe(p,n) and {+55}Mn(|a,3n).
 56FE2c  Production by {+54}Fe(|a,pn) and {+56}Fe(|a,p3n) (1996La20).
 56FE c  {+56}Co |e decay has been recommended as calibration standards for E|g
 56FE2c  and I|g see 1983LoZV, 1990Me15, 1990Ni03, 1990Tr06, 1991BaZS, 1999He10.
 56FE c  1990Me15: measured E|g, I|g for multi |g-ray calibration standards.
 56FE c  1988Wa26: measured E|g.
 56FE c  1986Br01: measured |g(|q) from oriented nuclei.
 56FE c  1984Oh03: measured |g(|q) from oriented nuclei.
 56FE c  1980St20: measured E|g, I|g, |g|g(|q), |g|g-coin with Ge(Li).
 56FE2c  Recalculated |a(K)exp from 1965Pe18.
 56FE c  1980Sh28: measured E|g, I|g, |g|g-coin and |g|g(|q).
 56FE c  1992ScZZ: measured |g-ray emission probabilities.
 56FE c  2008Dr04: measured I|g with HPGe.
 56FE c  2005MaZS: evaluation of |g-ray emission probabilities
 56FE2c  in the decay of {+56}Co.
 56FE c  2006BeZL: evaluation and recommendation of decay data
 56FE2c  for radionuclides.
 56FE c  Adopted decay scheme is taken mainly from 1980St20
 56FE cE           |b-measurement: E|b+=1464 keV {I15}(I|b|>90%);
 56FE2cE E|b+=440 keV {I30}(I|b|<10%) (1961Ha16); E|b+=1459 keV {I2}
 56FE3cE (I|b=18.1%), E|b+=440 keV (I|b|?0.8%) (1965Pe18).
 56FE cE LOGFT(L)  Value too low for a second-forbidden non-unique
 56FE2cE transition. Direct feeding to this level is expected to
 56FE3cE be negligible.
 56FE cG           Experimental internal conversion coefficients are
 56FE2cG calculated from Ice of 1965Pe18 and adopted I|g, normalizing
 56FE3cG to |a(K)=0.00026 for the 847|g (1980St20).
 56FE cG           For other |g-measurements, see 1967Hj01, 1967Ma03, 1968Gu05,
 56FE2cG 1969Au09, 1970Ph01, 1971Ag04, 1971Ca14, 1971Ge07, 1971Ge08, 1971Ta18,
 56FE3cG 1974HeYW, 1974Ho25, 1975Ka06, 1975Mc07, 1977Ge12, 1979Gr01, 1979Sh16,
 56FE4cG 1980Yo05, 1982Gr10, 1983Me17, 1983Mo20, 1996La20. For |g|g-coincidence
 56FExcG see 1980St20.
 56FE cG E         From 2006BeZl.
 56FE cG RI        From weighted average of values of 2006BeZL and 2008Dr04.
 56FE cG M         From 1984Oh03, based on |g(|q) from oriented nuclei,
 56FE2cG except as noted.
 56FE cG M(Y)      Based on |a(K)exp (1980St20).
 56FE cG M(Z),MR(W)$From adopted |g radiations.
 56FE cG M(X),MR(B)$From 1980Sh28.
 56FE cG MR        From 1986Br01, based on |g(|q) from oriented nuclei,
 56FE2cG except as noted.
 56FE cG MR(O)     From 1984Oh03.
 56FE cL E         From E|g and decay scheme using least-squares
 56FE2cL adjustment procedure.
 56FE cL J(X)      Consistent with |g(|q) from oriented nuclei (1984Oh03)
 56CO  P 0.0          4+               77.236 D  26             4566.0    20
 56FE  N 0.999399  23          1.0       1.0
 56FE cN NR        Based on assing no |e + |b{++} decay to the ground state and
 56FE2cN |s(I(|g+ce)(gs)=100, see 2005MaZS.
 56FE  L 0.0          0+
 56FE  L 846.7778  19 2+
 56FE  E               0.25  17  0.005 3   11.7   3             0.25      17
 56FES E EAV=1205.8 10 $CK=0.01753 4 $CL=0.001860 4 $CM+=0.0003249 8
 56FE  G 846.770   2  100      E2                                            C
 56FE4 G EKC=0.00026 (1980ST20)
 56FE  L 2085.1045 25 4+
 56FE  E               18.4  14  2.44  19  8.62   4             20.8      16
 56FES E EAV=631.2 9 $CK=0.1044 4 $CL=0.01109 5 $CM+=0.001938 8
 56FE  G 1238.288  3  66.50  12E2                                           YC
 56FE4 G EKC=0.000101 2 (1980ST20)
 56FE  L 2657.5894 25 2+
 56FE  G 1810.757  4  0.640  3 M1+E2     -0.17   3                           C
 56FE3 G FLAG=OZ$
 56FE  G 2657.527  4  0.0190 17
 56FE  L 2959.972   4 2+
 56FE  E              0.0067 13  0.018 4   10.37  9             0.025      5
 56FES E EAV=247.1 9 $CK=0.6496 21 $CL=0.06923 23 $CM+=0.01210 4
 56FE  G 2113.135  5  0.377  3 M1+E2     +0.27   3                           C
 56FE3 G FLAG=OZ$
 56FE  L 3122.970   3 4+
 56FE  E               1.041 20  9.01  6   7.580  4              10.05    6
 56FES E EAV=178.7 9 $CK=0.7966 15 $CL=0.08497 16 $CM+=0.01485 3
 56FE  G 1037.843  4  14.06   4 M1(+E2)   0.00    5                          C
 56FE3 G FLAG=BY$
 56FE4 G EKC=0.000133 10 (1980ST20)
 56FE cG MR        from 1038-1238 cascade (1980Sh28).
 56FE2cG Others: 0.00 {I4} from 1038-(1238)-847 cascade (1980Sh28),
 56FE3cG -0.02 {I2} (1974Ho25), +0.01 {I4} (1971Ag04), +0.02 {I2} (1971Ta18),
 56FE4cG -0.003 {I10} (1986Br01).
 56FE  G 2276.131  4  0.118  4 E2                                           ZC
 56FE  L 3369.95    7 2+
 56FE  E              6.0E-5 20  0.015  5  10.19 15             6.0E-5    20L
 56FES E EAV=76.7 9 $CK=0.8849 $CL=0.09456 $CM+=0.01653
 56FE  G 2523.09   11 0.059  4 M1+E2     +0.25   15                          C
 56FE3 G FLAG=ZW$
 56FE  G 3369.86   11 0.0101 7
 56FE  L 3445.348   3 3+
 56FE  E              0.0082 8   21.9  4   6.974  8              21.9     4
 56FES E EAV=45.3 9 $CK=0.8881 $CL=0.09496 $CM+=0.01660
 56FE  G 787.743   5  0.311  3 M1+E2     +0.85   35                          C
 56FE3 G FLAG=Y
 56FE4 G EKC=0.00027 3 (1980ST20)
 56FE  G 1360.212  4   4.286 12M1+E2     -0.11   1
 56FE3 G EKC=0.000076 2 (1980ST20)
 56FE cG MR        others: -0.12 {I5} (1980Sh28), -0.136 {I13} (1974Ho25),
 56FE2cG -0.11 {I2} (1971Ta18), -0.11 {I2} (1971Ag04), -0.116 {I+13-12}
 56FExcG (1984Oh03).
 56FE  G 2598.500  4  16.98  4 M1+E2     -0.28   2                          YC
 56FE3 G EKC=0.000026 1 (1980ST20)
 56FE cG MR        others: -0.27 {I+5-4} (1980Sh28), -0.266 {I16} (1974Ho25),
 56FE2cG -0.28 {I3} (1971Ta18), -0.278 {I11} (1971Ag04). -0.27 {I+9-12}
 56FExcG (1984Oh03).
 56FE  L 3856.495   3 3+
 56FE  E                         16.85  5  6.687  3              16.85     5
 56FES E  CK=0.8877 $CL=0.09555 $CM+=0.01672
 56FE  G 411.145   4  0.024  3
 56FE  G 486.55    11 0.054  2
 56FE  G 733.514   4  0.191  3 M1+E2     -0.02   2
 56FE3 G FLAG=Y$
 56FE4 G EKC=0.00025 6 (1980ST20)
 56FE  G 896.510   6  0.073  3
 56FE  G 1198.888  5  0.049  5                                               C
 56FE  G 1771.357  4  15.42  6 M1+E2     -0.004  +5-2                       YC
 56FE3 G EKC=0.0000472 18 (1980ST20)
 56FE cG MR        others: 0.00 {I3} (1980Sh28), 0.00 {I2} (1980Sh28),
 56FE2cG -0.01 {I1} (1974Ho25), -0.02 {I1} (1971Ta18), -0.023 {I8} (1971Ag04),
 56FE3cG -0.022 {I+5-6} (1984Oh03).
 56FE  G 3009.645   4 1.037  13 M1+E2    +0.065  5                           C
 56FE3 G EKC=0.000026 7 (1980ST20)
 56FE4 G FLAG=Y$
 56FE cG MR        others: <+0.25 (1974Ho25), +0.05 {I5} or
 56FE2cG -4.8 {I+12-19} (1971Ag04), 0.042 {I+35-36} (1984Oh03).
 56FE  L 4048.888   6 3+
 56FE  E                         3.965 13  7.038  4
 56FES E  CK=0.8871 $CL=0.09612 $CM+=0.01683
 56FE  G 1088.894  9  0.055  4 M1+E2     +0.43   12
 56FE  G 1963.741  8  0.707  4 M1+E2     +0.22   3
 56FE3 G EKC=0.000039 2 (1980ST20)
 56FE cG MR        others: +0.23 {I7} or +8.6 {I+10-31} (1984Oh03),
 56FE2cG +0.163 {I54} (74h025), +0.22 {I7} (1980Sh28), +0.19 {I6} (1971Ag04).
 56FE  G 3202.029  8  3.211  12M1+E2     +0.50   1                          YC
 56FE3 G EKC=0.000021 1 (1980ST20)
 56FE cG MR        others: +0.43 {I6} (1974Ho25), +0.51 {I6} (1971Ag04),
 56FE2cG +0.46 {I9} (1980Sh28), +0.524 {I+20-19} (1984Oh03).
 56FE  L 4100.363   3 4+               55 FS     25                         X
 56FE cL T         from DSA (1981Mu05).
 56FE  E                         12.66 4   6.442  4
 56FES E  CK=0.8868 $CL=0.09635 $CM+=0.01688
 56FE  G 655.003   5  0.043  4
 56FE  G 977.372   5  1.422  6 M1(+E2)   +0.07   +3-2
 56FE3 G FLAG=Y$
 56FE4 G EKC=0.000139 5 (1980ST20)
 56FE cG MR        others: +1.17 {I42} or -0.09 {I18} (1971Ag04),
 56FE2cG +0.12 {I1} (1980Sh28), +0.15 {I11} (1974Ho25). +0.077 {I+81-64}
 56FExcG (1984Oh03).
 56FEB G BM1W=(0.048 22)$BE2W=(0.5 5)
 56FE  G 1140.368  6  0.132  3                                               C
 56FE  G 1442.746  6  0.180  4                                               C
 56FE  G 2015.215  5  3.018  12 M1+E2     +0.68   5                         Y
 56FE cG MR        other: +0.74 {I+5-6} (1984Oh03).
 56FE3 G EKC=0.000039 2 (1980ST20)
 56FEB G BM1W=0.008 4$BE2W=1.8 9
 56FE  G 3253.503  4  7.928  21E2                                            C
 56FE3 G EKC=0.0000195 9 (1980ST20)
 56FEB G BE2W=1.4 7
 56FE  L 4119.936   3 3+                                                    X
 56FE  E                         9.940 18  6.509  4
 56FES E  CK=0.8867 $CL=0.09645 $CM+=0.01690
 56FE  G 263.434   5  0.0220 3
 56FE  G 674.570   5  0.024  3
 56FE  G 996.948   5  0.111  4 M1+E2
 56FE cG MR        -0.34 {I+14-11} or -2.1 {I6} (1986Br01).
 56FE  G 1159.944  6  0.094  6 M1+E2     +0.064  +16-36                      C
 56FE  G 1462.322  6  0.074  4                                               C
 56FE  G 2034.791  5  7.774  28M1+E2     -0.073  5                           C
 56FE cG MR        others: -0.087 {I12} (1974Ho25), -0.065 {I8} (1971Ag04),
 56FE2cG -0.0070 {I+9-8} (1984Oh03).
 56FE3 G EKC=0.0000372 13 (1980ST20)
 56FE  G 3273.079  4  1.877  2 M1+E2     +0.420  4                          YC
 56FE cG MR        others: 0.37 {I6} or 1.56 {I20} (1974Ho25),
 56FE2cG +0.41 {I6} or +1.63 {I20} (1971Ag04), +0.430 {I21} (1984Oh03).
 56FE4 G EKC=0.0000191 17 (1980ST20)
 56FE  L 4298.096   3 4+               110 FS    50
 56FE cL T         from DSA (1981En03).
 56FE cL J         J=4 from |g(|q) in 1986Br01.
 56FE  E                         3.688 13  6.488  7
 56FES E  CK=0.8847 $CL=0.09810 $CM+=0.01722
 56FE  G 852.732   4  0.049  3
 56FE  G 1175.101  4  2.253  6 M1+E2     +0.14   4                          X
 56FE4 G EKC=0.000094 21 (1980ST20)
 56FE cG MR        others: +0.41 {I+7-8} or -0.33 {I9} (1980Sh28),
 56FE2cG -1,70 {I+26-33} (74h025), +1.6 {I3} or -0.25 {I10} (1971Ag04).
 56FEB G BM1W=0.07 4$BE2W=2.1 16
 56FE  G 1640.475  5  0.0616 19                                              C
 56FE  G 2212.944  4  0.388  4 M1+E2      -3.0   10
 56FE cG E         weighted average of values of 1988Wa26 and 1979He19.
 56FE cG MR        others: -2.0 {I+6-10} 0R 0.30 {I+19-15} (74h025),
 56FE2cG +2.6 {I+20-10} or -0.5 {I2} (1971Ag04), -2.8 {I+13-11} (1984Oh03).
 56FEB G BM1W=0.00019 15$BE2W=0.7 4
 56FE  G 3451.232  4  0.950  5 E2                                            C
 56FE3 G EKC=0.0000113 20 (1980ST20)
 56FE cG EKC       Others: 0.0000116 {I21} or 0.0000147 {I15} (1976MeZM)
 56FEB G BE2W=0.21 10
 56FE  L 4394.93    5 3+
 56FE  E                        0.2159 18  7.319 12
 56FES E  CK=0.8818 $CL=0.1005 $CM+=0.01770
 56FE  G 1271.92   6  0.0200 7
 56FE  G 3548.05   6  0.1956 15M1+E2     -0.30   2                           C
 56FE cG MR        others: -0.18 {I14} or >6.7 (74h025),
 56FE2cG -0.26 {I20} or <-3.7 (1971Ag04), -0.228 {I+46-42} (1984Oh03).
 56FE  L 4447.7     4(2-,3,4)
 56FE  E                        0.0167  5  8.095 21
 56FES E  CK=0.8781 $CL=0.10361 19 $CM+=0.01832 4
 56FE  G 3600.8    4  0.0167 5                                               C
 56FE  L 4458.406  184+
 56FE  E                         0.209 7   6.910 23
 56FES E  CK=0.8768 $CL=0.10464 22 $CM+=0.01853 5
 56FE  G 1335.40   3  0.1225 12                                              C
 56FE  G 2373.24   3  0.080  4
 56FE  G 3611.53   3  0.0086 3                                               C"""
    nuclide = "Co56"
    result = get_json_from_ensdf(ensdf_data, nuclide)
    assert isinstance(result, dict)
    assert result["parent"]["a"] == "56"
    assert result["daughter"]["element"] == "Fe"
    assert len(result["radiation"]) == 4

In [14]:
ipytest.run()

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

..............                                                                               [100%]
========================================= warnings summary =========================================
../../../../../miniconda3/envs/tardis/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:302
  /home/sherryff/miniconda3/envs/tardis/lib/python3.8/site-packages/ipywidgets/widgets/widget.py:302: DeprecationWarning: Widget registration using a string name has been deprecated. Widget registration now uses a plain `@register` decorator.
    warnings.warn("Widget registration using a string name has been deprecated. Widget registration now uses a plain `@register` decorator.", DeprecationWarning)

../../../../../miniconda3/envs/tardis/lib/python3.8/site-packages/qgrid/grid.py:573
  /home/sherryff/miniconda3/envs/tardis/lib/python3.8/site-packages/qgrid/grid.py:573: DeprecationWarning: metadata {'sync': True} was set from the constructor. With traitlets 4.1, metadata should be set using 

<ExitCode.OK: 0>